<a href="https://colab.research.google.com/github/rohil9/spice-grill/blob/main/Nengo_MNIST_example3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Nengo Benchmark Model: MNIST
Input: Images from the MNIST data set
Ouput: the categorization of the image
"""
"""
This code has been updated by Jose Domingue-jodom4@morgan.edu,
Rober Hill-rohill9@morgan.edu, Maryline Happy -mahap1@morgan.edu
"""
!pip install nengo
!pip install pytry

import nengo
from nengo.neurons import LIF
import numpy as np
import pytry
import timeit
import sklearn.datasets

class MNIST(pytry.NengoTrial):
    def params(self):
        self.param('number of neurons', n_neurons=500)
        self.param('time per image', t_image=0.1)
        self.param('training samples', n_training=5000)
        self.param('testing samples', n_testing=100)
        self.param('output synapse', synapse=0.02)
        self.param('use Gabor encoders', use_gabor=False)
        self.param('iterations of backprop to run', n_backprop=0)
        self.param('learning rate', learning_rate=1e-3)
        self.param('iterations of feedback alignment to run', n_feedback=0)
        self.param('regularization', reg=0.1)
        


    def model(self, p):
        mnist = sklearn.datasets.fetch_openml('mnist_784')

        x = mnist['data'].astype(float)-128
        x = x/np.linalg.norm(x, axis=1)[:,None]
        y = mnist['target']
        y = np.eye(10)[y.astype(int)]*2-1
        y = y/np.linalg.norm(y, axis=1)[:,None]

        order = np.arange(len(x))
        np.random.shuffle(order)
        x = x[order]
        y = y[order]
        
        model = nengo.Network() #nengo.neurons.NeuronType=nengo.LIF
        with model:
            def stim_func(t, data=x[p.n_training:]):
                index = int(t / p.t_image)
                return data[index % len(data)]
            stim = nengo.Node(stim_func)

            if p.use_gabor:
                from nengo_extras.vision import Gabor, Mask
                encoders = Gabor().generate(p.n_neurons, (11, 11))
                encoders = Mask((28, 28)).populate(encoders, flatten=True)
            else:
                encoders = nengo.dists.UniformHypersphere(surface=True)

            ens = nengo.Ensemble(n_neurons=p.n_neurons, dimensions=784,neuron_type=LIF(),
                                 encoders=encoders,
                                 intercepts=nengo.dists.CosineSimilarity(784+2))
            
            def result_func(t, raw, correct=y[p.n_training:]):
                index = int((t-p.dt) / p.t_image)
                c = np.argmax(correct[index])
                r = np.argmax(raw)
                return np.hstack([raw, c, r])
            result = nengo.Node(result_func, size_in=10)

            nengo.Connection(stim, ens, synapse=None)
            c = nengo.Connection(ens, result, 
                                 eval_points=x[:p.n_training], 
                                 function=y[:p.n_training],
                                 solver=nengo.solvers.LstsqL2(reg=p.reg),
                                 synapse=p.synapse)

            self.p_output = nengo.Probe(result)

        if p.n_backprop > 0:
            import nengo_encoder_learning as nel
            nel.improve(c, backprop=True,
                        learning_rate=p.learning_rate, 
                        steps=p.n_backprop,
                        seed=np.random.randint(0x7FFFFFFF))
        if p.n_feedback > 0:
            import nengo_encoder_learning as nel
            nel.improve(c, backprop=False,
                        learning_rate=p.learning_rate, 
                        steps=p.n_feedback,
                        seed=np.random.randint(0x7FFFFFFF))
        return model


    def evaluate(self, p, sim, plt):
        start = timeit.default_timer()
        T = p.n_testing * p.t_image
        sim.run(T)
        end = timeit.default_timer()
        speed = T / (end - start)

        correct = sim.data[self.p_output][:,10].astype(int)
        result = sim.data[self.p_output][:,11].astype(int)
        confusion = np.zeros((10,10), dtype=int)
        count = np.zeros(10, dtype=int)
        times = sim.trange()
        for i in range(p.n_testing):
            t = (i + 1) * p.t_image
            index = np.argmax(times >= t) - 1
            count[correct[index]] += 1
            confusion[correct[index],result[index]] += 1

        score = sum(confusion[i, i] for i in range(10)) / float(p.n_testing)

        if plt is not None:
            plt.subplot(2, 1, 1)
            plt.plot(sim.trange(), sim.data[self.p_output][:,:10])
            plt.xlim(0, p.t_image * min(p.n_testing, 20))
            plt.ylabel('output')
            plt.subplot(2, 1, 2)
            plt.plot(sim.trange(), sim.data[self.p_output][:,10:])
            plt.legend(['correct', 'chosen'], loc='best')
            plt.xlim(0, p.t_image * min(p.n_testing, 20))

        return dict(speed=speed, score=score, count=count, confusion=confusion)


     |████████████████████████████████| 399kB 9.3MB/s 
  Created wheel for pytry: filename=pytry-0.9.2-cp36-none-any.whl size=8726 sha256=fa313bcff1a879f1ab87d6e19b5c9aee79781f3e76cd956eb12b6c91ddabaf68
  Stored in directory: /root/.cache/pip/wheels/b5/d1/78/a6795785fa94f84b41b1bb9fd175454a1214cbd37ef73eb005
Successfully built pytry


In [ ]:
MNIST().run()#verbose=False

running MNIST#20201112-055224-f6b7f953


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

seed = 1
backend = 'nengo'
dt = 0.001
neuron_type = 'default'
n_neurons = 500
t_image = 0.1
n_training = 5000
n_testing = 100
synapse = 0.02
use_gabor = False
n_backprop = 0
learning_rate = 0.001
n_feedback = 0
reg = 0.1
confusion = array([[13,  0,  0,  1,  0,  0,  0,  0,  0,  0],
       [ 0, 17,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  2,  1,  0,  0,  0,  1,  0,  0],
       [ 0,  1,  0,  2,  0,  1,  0,  0,  0,  0],
       [ 0,  1,  0,  1, 10,  0,  0,  0,  0,  1],
       [ 2,  0,  0,  1,  0,  3,  0,  1,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  9,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  9,  0,  0],
       [ 0,  1,  2,  1,  0,  0,  0,  0,  8,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  2,  0,  6]])
count = array([14, 17,  5,  4, 13,  8,  9, 10, 12,  8])
score = 0.79
speed = 1.6601123138792608


{'confusion': array([[13,  0,  0,  1,  0,  0,  0,  0,  0,  0],
        [ 0, 17,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  0,  2,  1,  0,  0,  0,  1,  0,  0],
        [ 0,  1,  0,  2,  0,  1,  0,  0,  0,  0],
        [ 0,  1,  0,  1, 10,  0,  0,  0,  0,  1],
        [ 2,  0,  0,  1,  0,  3,  0,  1,  1,  0],
        [ 0,  0,  0,  0,  0,  0,  9,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  1,  0,  9,  0,  0],
        [ 0,  1,  2,  1,  0,  0,  0,  0,  8,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  2,  0,  6]]),
 'count': array([14, 17,  5,  4, 13,  8,  9, 10, 12,  8]),
 'score': 0.79,
 'speed': 1.6601123138792608}